<a href="https://colab.research.google.com/github/wwdz540/AI-TEST/blob/main/%E2%80%9CQuickStart_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c30c1dcf2bc20119bcda7e734ce0eb42/quickstart_tutorial.ipynb#scrollTo=9eMtlafIw09I

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


import torch

torch.__version__
print(torch.version.cuda)




11.3


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
training_data  = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader  = DataLoader(test_data,batch_size=batch_size)


for X,y in test_dataloader : 
    print(f'Shape of X X[N, C, H ,W]: {X.shape}')
    print(f'Shape of y: {y.shape}')
    break



Shape of X X[N, C, H ,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [ ]:
 torch.cuda.is_available() 

False

In [ ]:
#torch.cuda.is_available()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using {device} device')

Using cuda device


In [ ]:
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)            
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    
model = NeuralNetwork().to(device)
print(model)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)





In [ ]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        pred = model.forward(X)
        loss = loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0 :
            loss , current = loss.item(), batch * len(X)
            print(f'loss {loss:>7f} [{current:>5d} / {size:>5d}]')
            

In [ ]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss , correct = 0 , 0 
    with torch.no_grad():
        for X , y in dataloader:
            X , y = X.to(device) , y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
    test_loss /= num_batches
    correct /= size
    print(f'Test Error :\n Accuracy : {(100 * correct):>0.1f}% , Avg loss {test_loss:>8f} \n')


In [ ]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t + 1}\n ------------------------------------')
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done")


Epoch 1
 ------------------------------------
loss 2.308282 [    0 / 60000]
loss 2.294305 [ 6400 / 60000]
loss 2.272617 [12800 / 60000]
loss 2.269747 [19200 / 60000]
loss 2.235807 [25600 / 60000]
loss 2.224751 [32000 / 60000]
loss 2.223505 [38400 / 60000]
loss 2.191728 [44800 / 60000]
loss 2.191579 [51200 / 60000]
loss 2.156755 [57600 / 60000]
Test Error :
 Accuracy : 43.1% , Avg loss 2.154829 

Epoch 2
 ------------------------------------
loss 2.165566 [    0 / 60000]
loss 2.153264 [ 6400 / 60000]
loss 2.097365 [12800 / 60000]
loss 2.120242 [19200 / 60000]
loss 2.041207 [25600 / 60000]
loss 2.008408 [32000 / 60000]
loss 2.026018 [38400 / 60000]
loss 1.949537 [44800 / 60000]
loss 1.957033 [51200 / 60000]
loss 1.890529 [57600 / 60000]
Test Error :
 Accuracy : 50.5% , Avg loss 1.886860 

Epoch 3
 ------------------------------------
loss 1.922072 [    0 / 60000]
loss 1.887555 [ 6400 / 60000]
loss 1.776109 [12800 / 60000]
loss 1.826244 [19200 / 60000]
loss 1.683432 [25600 / 60000]
loss 1